In [1]:
import requests as r
import os
import pandas as pd
from pandas import json_normalize
import json

In [2]:
#Schema das Tabelas

data_types_dict_orcamentaria = {'tipo_matriz': 'str', 'cod_ibge': 'str', 'classe_conta': 'int', 'conta_contabil': 'str', 'poder_orgao': 'int', 'ano_fonte_recursos': 'int', 'fonte_recursos': 'str', 'funcao': 'int', 'subfuncao': 'int', 'exercicio': 'int', 'mes_referencia': 'int', 'educacao_saude': 'int', 'data_referencia': 'str', 'entrada_msc': 'int', 'natureza_despesa': 'str', 'ano_inscricao': 'int', 'natureza_receita': 'str', 'valor': 'float', 'natureza_conta': 'str', 'tipo_valor': 'str', 'complemento_fonte': 'int'}
data_types_dict_patrimonial = {'tipo_matriz': 'str', 'cod_ibge': 'str', 'classe_conta': 'int', 'conta_contabil': 'str', 'poder_orgao': 'int', 'financeiro_permanente': 'int', 'ano_fonte_recursos': 'int', 'fonte_recursos': 'int', 'exercicio': 'int', 'mes_referencia': 'int', 'divida_consolidada': 'int', 'data_referencia': 'str', 'entrada_msc': 'int', 'valor': 'float', 'natureza_conta': 'str', 'tipo_valor': 'str', 'complemento_fonte': 'int'}
data_types_dict_controle = {'tipo_matriz': 'str', 'cod_ibge': 'str', 'classe_conta': 'int', 'conta_contabil': 'str', 'poder_orgao': 'int', 'ano_fonte_recursos': 'int', 'fonte_recursos': 'str', 'funcao': 'int', 'subfuncao': 'int', 'exercicio': 'int', 'mes_referencia': 'int', 'educacao_saude': 'int', 'data_referencia': 'str', 'entrada_msc': 'int', 'natureza_despesa': 'str', 'ano_inscricao': 'int', 'valor': 'float', 'natureza_conta': 'str', 'tipo_valor': 'str', 'complemento_fonte': 'int'}

In [3]:
#função para juntar todos os arquivos de classe
def merge(arquivo:str, arquivos:list, path_de_saida:str,data_types_dict:dict)->None:
    df = pd.read_csv(arquivo, sep=';')
    os.remove(arquivo)
    for parte in arquivos:
        df2 = pd.read_csv(parte, sep=';')
        if(not(df2.empty)):
            df = pd.concat([df, df2], ignore_index=True)
            os.remove(parte)
    df = df.fillna(-1)
    df = df.astype(data_types_dict)
    df['valor'] = df['valor'].apply(lambda x: round(x,2))
    df.to_csv(path_de_saida,sep =';', decimal=',', index=False)
    
    
def listar_path(comeco:str,arquivos_csv:str):
    lista_caminho = []
    for subdir, _, files in os.walk(arquivos_csv,topdown=True):
               for file in files:
                    if file.startswith(comeco) and file.endswith('.csv'):
                        caminho_dos_arquivos =  subdir + "\\" + file
                        lista_caminho.append(caminho_dos_arquivos)
    return lista_caminho


In [6]:
#Paritmonial
periodo_lista = ['beginning_balance','period_change','ending_balance']
for i in range(1,5):
    for periodo in periodo_lista:
        url = f'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_patrimonial?id_ente=3301009&an_referencia=2023&me_referencia=01&co_tipo_matriz=MSCC&classe_conta={i}&id_tv={periodo}'
        resp = r.get(url)
        data = json.loads(resp.content)
        if(not(data["items"] == [])):
            arquivo = f'F:\\CIC\\17 - Estagiários\\Rafael_Pereira\\MSC\\msc_patrimonial_classe{i}_{periodo}.csv'
            df = json_normalize(data["items"])
            df.to_csv(arquivo,sep =';', index=False)

In [7]:
arquivos = 'F:\\CIC\\17 - Estagiários\\Rafael_Pereira\\MSC'
lista = listar_path('msc_patrimonial',arquivos)
merge(lista[0],lista[1:],'F:\\CIC\\17 - Estagiários\\Rafael_Pereira\\MSC\\msc_patrimonial.csv',data_types_dict_patrimonial)

In [10]:
#Orçamentario
for i in range(5,7):
    for periodo in periodo_lista:
        url = f'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_orcamentaria?id_ente=3301009&an_referencia=2023&me_referencia=01&co_tipo_matriz=MSCC&classe_conta={i}&id_tv={periodo}'
        resp = r.get(url)
        data = json.loads(resp.content)
        if(not(data["items"] == [])):
            arquivo = f'F:\\CIC\\17 - Estagiários\\Rafael_Pereira\\MSC\\msc_orcamentaria_classe{i}_{periodo}.csv'
            df = json_normalize(data["items"])
            df.to_csv(arquivo,sep =';', index=False)

In [11]:
lista = listar_path('msc_orcamentaria',arquivos)
merge(lista[0],lista[1:],'F:\\CIC\\17 - Estagiários\\Rafael_Pereira\\MSC\\msc_orcamentaria.csv',data_types_dict_orcamentaria)

In [12]:
#controle
for i in range(7,9):
    for periodo in periodo_lista:
        url = f'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_controle?id_ente=3301009&an_referencia=2023&me_referencia=01&co_tipo_matriz=MSCC&classe_conta={i}&id_tv={periodo}'
        resp = r.get(url)
        data = json.loads(resp.content)
        if(not(data["items"] == [])):
            arquivo = f'F:\\CIC\\17 - Estagiários\\Rafael_Pereira\\MSC\\msc_controle_classe{i}_{periodo}.csv'
            df = json_normalize(data["items"])
            df.to_csv(arquivo,sep =';', index=False)

In [13]:
lista = listar_path('msc_controle_',arquivos)
merge(lista[0],lista[1:],'F:\\CIC\\17 - Estagiários\\Rafael_Pereira\\MSC\\msc_controle.csv',data_types_dict_controle)